In [1]:
library(tidyverse)
library(repr)
library(RColorBrewer)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [5]:
set.seed(19)
pulsar_data <- read_csv("https://github.com/kseniak1/DSCI100-Group-Project/raw/main/HTRU_2%5B1%5D.csv", col_names = FALSE) %>%
                rename(mean_ip = X1,
                      dev_ip = X2,
                      excess_ip = X3,
                      skewness_ip = X4,
                      mean_dm = X5,
                      dev_dm = X6,
                      excess_dm = X7,
                      skewness_dm = X8,
                      class = X9) %>%
                mutate(class = as.factor(class)) %>%
                group_by(class) %>%
                sample_n(size = 250)
                
pulsar_data

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double(),
  X9 = col_double()
)



mean_ip,dev_ip,excess_ip,skewness_ip,mean_dm,dev_dm,excess_dm,skewness_dm,class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
115.23438,46.13541,0.439600186,0.541190958,2.4205686,14.283309,8.962998,108.34897,0
125.64844,55.50557,-0.017358015,-0.641545522,2.0602007,15.117369,10.893795,141.26112,0
123.68750,49.31797,-0.002264451,0.218708063,6.7307692,31.654744,5.074507,26.22959,0
113.00781,43.74152,0.491563810,0.463688081,4.0008361,20.109961,6.990697,58.85479,0
129.40625,53.41515,-0.072911778,-0.349883263,2.7215719,20.161022,8.202760,71.51378,0
127.28906,49.84604,0.024368056,-0.315295506,3.7734114,18.704479,6.539690,52.38843,0
123.38281,41.98086,0.232734458,0.691077029,1.8821070,12.464544,11.334179,175.53199,0
109.92188,39.79176,0.261571176,1.002609883,1.8653846,16.782131,10.572529,121.75195,0
96.88281,47.55993,0.735227976,0.767092927,2.4264214,17.545129,9.097817,95.07288,0
